# 第三方库

## 安装

使用包管理工具：`pip`, `conda` 之类的
- `pip install <包名>`

## PyInstaller

将.py源代码转换成无需源代码的可执行文件

安装：`pip install pyinstaller`

在命令行工具中使用：cmd or powershell or terminal
- `$ pyinstaller -F <文件名.py>`

参数 | 描述
:--: | --
-h |查看帮助
--clean | 清理打包过程中的临时文件
-D, --onedir | 默认值，生成dist文件夹
-F, --onefile | 在dist文件夹中只生成独立的打包文件
-i <图标.ico> | 指定打包程序使用的图标(icon)文件

tip:
命令行工具不会用：直接 `工具 --help` 或 `工具 help`


## jieba库

- 优秀的中文分词第三方库
- 提供三种分词模式，最简单只需掌握一个函数

安装：`pip install jieba`

### 分词的原理

依靠中文词库
- 利用一个中文词库，确定中文字符之间的关联概率
- 中文字符间概率大的组成词组，形成分词结果
- 除了分词，用户还可以添加自定义的词组

### 三种模式

- 精确模式：把文本精确的切分开，不存在冗余单词 `jieba.lcut(str)`
- 全模式：把文本中所有可能的词语都扫描出来，有冗余 `jieba.lcut(s,
cut_all=True)`
- 搜索引擎模式：在精确模式基础上，对长词再次切分 `jieba.lcut_for_sear
ch(s)`

In [1]:
import jieba

In [2]:
jieba.lcut("""说实话，我不喜欢Python这种弱类型语言，反而我更喜欢C#、Rust这些强类型语言，编译器就能为我们解决很多问题。
但脚本语言也有存在的意义，就是用来写脚本的。""")[:5]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Cheng\AppData\Local\Temp\jieba.cache
Loading model cost 0.742 seconds.
Prefix dict has been built successfully.


['说实话', '，', '我', '不', '喜欢']

### 应用 词频分析

In [3]:
import jieba

txt = open("assets/三国演义.txt", "r", encoding="utf-8").read()
words = jieba.lcut(txt)
counts = {}
for word in words:
    if len(word) == 1:
        continue
    else:
        counts[word] = counts.get(word,0) + 1 # 不存在则为0

items = list(counts.items()) # 转换成 (词语, 频率) 元组的列表
items.sort(key=lambda x:x[1], reverse=True) # 从大到小排序

# 打印
for i in range(15):
    word, count = items[i]
    print("{0:5}\t{1:>5}".format(word, count))

曹操   	  953
孔明   	  836
将军   	  772
却说   	  656
玄德   	  585
关公   	  510
丞相   	  491
二人   	  469
不可   	  440
荆州   	  425
玄德曰  	  390
孔明曰  	  390
不能   	  384
如此   	  378
张飞   	  358


In [4]:
# 人名统计 要去除非人名，合并同一个人

import jieba
txt = open("assets/三国演义.txt", "r", encoding="utf-8").read()
excludes = {"将军","却说","荆州","二人","不可","不能","如此","商议", "丞相", "主公", "军士", "如何", "左右", "军马", "引兵", "次日", "大喜", "东吴", "天下", "于是", "今日", "魏兵", "不敢", "一人", "陛下", "都督", "人马", "不知"}
txt = txt.replace("曹丞相", "曹操")
words = jieba.lcut(txt)
counts = {}

# 合并指向统一人物的词
for word in words:
    if len(word) == 1:
        continue
    elif word == "诸葛亮" or word == "孔明曰":
        rword = "孔明"
    elif word == "关公" or word == "云长":
        rword = "关羽"
    elif word == "玄德" or word == "玄德曰":
        rword = "刘备"
    elif word == "孟德":
        rword = "曹操"
    else:
        rword = word
    counts[rword] = counts.get(rword,0) + 1
for word in excludes: # 删除不要的词
    del counts[word]

items = list(counts.items())
items.sort(key=lambda x:x[1], reverse=True)
for i in range(10):
    word, count = items[i]
    print("{0:5}\t{1:>5}".format(word, count))

孔明   	 1383
刘备   	 1252
曹操   	  980
关羽   	  784
张飞   	  358
吕布   	  300
赵云   	  278
孙权   	  264
司马懿  	  221
周瑜   	  217


## wordcloud

- wordcloud是优秀的词云展示第三方库
- 空格分隔词，出现的多，词越大

安装：`pip install wordcloud`

生成 WordCloud 词云对象：
- 调用构造函数 `w = wordcloud.WordCloud()`
- 参数：
    - `width` 指定词云对象生成图片的宽度，默认400像素
    - `height` 指定词云对象生成图片的高度，默认200像素
    - `min_font_size` 指定词云中字体的最小字号，默认4号
    - `max_font_size` 指定词云中字体的最大字号，根据高度自动调节
    - `font_step` 指定词云中字体字号的步进间隔，默认为1
    - `font_path` 指定字体文件的路径，默认None
    - `max_words` 指定词云显示的最大单词数量，默认200
    - `stop_words` 指定词云的排除词列表，即不显示的单词列表
    - `mask` 指定词云形状，默认为长方形，需要引用imread()函数
        - `from scipy.misc import imread`
        - `mk=imread("pic.png")`
    - `background_color` 指定词云图片的背景颜色，默认为黑色

方法：
- `w.generate()`
- `w.to_file()`

In [5]:
# 还是以三国演义为例子吧

import jieba
import wordcloud
# pip install imageio
import imageio

mask = imageio.imread("assets/诸葛.jpg")

txt = open("assets/三国演义.txt", "r", encoding="utf-8").read()
w = wordcloud.WordCloud( width=1000, height=700,\
    font_path="msyh.ttc", mask=mask, background_color="white")

excludes = {"将军","却说","荆州","二人","不可","不能","如此","商议", "丞相", "主公", "军士", "如何", "左右", "军马", "引兵", "次日", "大喜", "东吴", "天下", "于是", "今日", "魏兵", "不敢", "一人", "陛下", "都督", "人马", "不知"}

words = [i for i in jieba.lcut(txt) if len(i) > 1 and i not in excludes]
# 不要一个字的和不想要的词

for i in range(len(words)):
    if words[i] == "诸葛亮" or words[i] == "孔明曰":
        words[i] = "孔明"
    elif words[i] == "关公" or words[i] == "云长":
        words[i] = "关羽"
    elif words[i] == "玄德" or words[i] == "玄德曰":
        words[i] = "刘备"
    elif words[i] == "孟德":
        words[i] = "曹操"

w.generate(" ".join(words))
w.to_file("output/三国演义词云.png")